In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter

In [2]:
dfs = []
submissions = ['submission_resnet50.csv', 'submission_se_resnet50.csv', 'submission_densenet201.csv']

n_submissions = len(submissions)

for submission in submissions:
    dfs.append(pd.read_csv(submission))

In [55]:
dfs[0].head()

,id,attribute_ids
0,10023b2cc4ed5f68,195 223 343 344 369 587 766 1059
1,100fbe75ed8fd887,93 231 1039
2,101b627524a04f19,79 784 1037
3,10234480c41284c6,147 483 725 738 776 813 830 1006 1046
4,1023b0e2636dcea8,147 322 477 584 813 954 1046 1092


In [116]:
df = pd.DataFrame(columns=dfs[0].columns)

In [117]:
dfs[0].shape[0]

7443

In [118]:
for i in range(dfs[0].shape[0]):
    attributes = np.array([])
    id = dfs[0].loc[i, 'id']
    attributes = np.concatenate([submission.loc[i, 'attribute_ids'].split(' ') for submission in dfs])
    unique, counts = np.unique(attributes, return_counts=True)
    attribute_ids = ' '.join(unique[counts > n_submissions // 2])
    data = [id, attribute_ids]
    tmp_df = pd.DataFrame([data], columns=df.columns)
    df = pd.concat([df, tmp_df])
    

In [119]:
df.head()

,id,attribute_ids
0,10023b2cc4ed5f68,1059 195 223 289 343 344 369 587 766
0,100fbe75ed8fd887,1039 231 93
0,101b627524a04f19,420 784 79
0,10234480c41284c6,1006 1046 147 725 738 776 813 830
0,1023b0e2636dcea8,1046 1092 147 322 584 776 813 954


In [125]:
df.to_csv('submission_ensemble_voting.csv', index=False)

In [121]:
df.shape

(7443, 2)

In [124]:
pd.read_csv('submission_ensemble_voting.csv').head()

,id,attribute_ids
0,10023b2cc4ed5f68,1059 195 223 289 343 344 369 587 766
1,100fbe75ed8fd887,1039 231 93
2,101b627524a04f19,420 784 79
3,10234480c41284c6,1006 1046 147 725 738 776 813 830
4,1023b0e2636dcea8,1046 1092 147 322 584 776 813 954


In [3]:
from sklearn.multioutput import MultiOutputRegressor